In [ ]:
import numpy as np
from typing import List, Dict, Any

def interval_vikor(
    matrix: np.ndarray,        # shape (m, n, 2) lower/upper
    weights: List[float],      # len n, sum = 1
    types: List[int],          # len n, 1 = benefit, -1 = cost
    v: float = 0.5
) -> Dict[str, Any]:
    """
    Интервальная реализация VIKOR (формулы по Sayadi et al.).
    Возвращает интервальные S, R, Q и ранжирование.
    """

    m, n = matrix.shape[0], matrix.shape[1]
    L = matrix[:, :, 0]
    U = matrix[:, :, 1]

    # 1) интервальные идеал/надир для каждого критерия
    f_star = np.zeros((n, 2))   # [low, high]
    f_minus = np.zeros((n, 2))  # [low, high]

    for j in range(n):
        if types[j] == 1:  # benefit
            f_star[j, 0] = np.max(L[:, j])   # best lower
            f_star[j, 1] = np.max(U[:, j])   # best upper
            f_minus[j, 0] = np.min(L[:, j])  # worst lower
            f_minus[j, 1] = np.min(U[:, j])  # worst upper
        else:  # cost
            f_star[j, 0] = np.min(L[:, j])
            f_star[j, 1] = np.min(U[:, j])
            f_minus[j, 0] = np.max(L[:, j])
            f_minus[j, 1] = np.max(U[:, j])

    # 2) вычисляем интервальные нормализованные разрывы D_ij = [D_low, D_high]
    D_low = np.zeros((m, n))
    D_high = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            fstar_low = f_star[j, 0]
            fstar_high = f_star[j, 1]
            fminus_low = f_minus[j, 0]
            fminus_high = f_minus[j, 1]

            over_ij = U[i, j]
            under_ij = L[i, j]

            denom_low = (fstar_low - fminus_high)
            denom_high = (fstar_high - fminus_low)

            # безопасное деление
            if abs(denom_low) < 1e-12:
                d_low = 0.0
            else:
                d_low = (fstar_low - over_ij) / denom_low

            if abs(denom_high) < 1e-12:
                d_high = 0.0
            else:
                d_high = (fstar_high - under_ij) / denom_high

            # нормируем в [0,1]
            D_low[i, j] = np.clip(d_low, 0.0, 1.0)
            D_high[i, j] = np.clip(d_high, 0.0, 1.0)

    # 3) агрегируем по весам: интервальные S_i и R_i
    w = np.array(weights)
    S = np.zeros((m, 2))  # [S_low, S_high]
    R = np.zeros((m, 2))  # [R_low, R_high]

    for i in range(m):
        S[i, 0] = np.sum(w * D_low[i, :])
        S[i, 1] = np.sum(w * D_high[i, :])
        R[i, 0] = np.max(w * D_low[i, :]) if n > 0 else 0.0
        R[i, 1] = np.max(w * D_high[i, :]) if n > 0 else 0.0

    # 4) вычисляем S*, S-, R*, R- (по компонентам интервалов)
    S_star_low = np.min(S[:, 0])
    S_star_high = np.min(S[:, 1])
    S_minus_low = np.max(S[:, 0])
    S_minus_high = np.max(S[:, 1])

    R_star_low = np.min(R[:, 0])
    R_star_high = np.min(R[:, 1])
    R_minus_low = np.max(R[:, 0])
    R_minus_high = np.max(R[:, 1])

    # безопасное деление-утилита
    def safe_div(numer, denom):
        return numer / denom if abs(denom) > 1e-12 else 0.0

    # 5) интервальный Q: левый/правый по формуле
    Q = np.zeros((m, 2))
    for i in range(m):
        # левый (пессимистичный) — используем S_low и комбинируем в соответствии с докладом
        num_S_left = S[i, 0] - S_star_high
        den_S_left = S_minus_low - S_star_high
        term_S_left = safe_div(num_S_left, den_S_left)

        num_R_left = R[i, 0] - R_star_high
        den_R_left = R_minus_low - R_star_high
        term_R_left = safe_div(num_R_left, den_R_left)

        q_left = v * term_S_left + (1.0 - v) * term_R_left

        # правый (оптимистичный)
        num_S_right = S[i, 1] - S_star_low
        den_S_right = S_minus_high - S_star_low
        term_S_right = safe_div(num_S_right, den_S_right)

        num_R_right = R[i, 1] - R_star_low
        den_R_right = R_minus_high - R_star_low
        term_R_right = safe_div(num_R_right, den_R_right)

        q_right = v * term_S_right + (1.0 - v) * term_R_right

        Q[i, 0] = np.clip(q_left, 0.0, 1.0)
        Q[i, 1] = np.clip(q_right, 0.0, 1.0)

    # 6) ранжирование (без центров) — лексикографически по (Q_left, Q_right)
    #       меньшие Q_left лучше; при равенстве Q_left — меньший Q_right лучше
    order = np.lexsort((Q[:, 1], Q[:, 0]))  # lexsort uses last key first; here keys=(Q_right, Q_left)
    ranking = order.tolist()

    # 7) условия приемлемости (интервальные версии)
    sorted_Q = Q[order]
    DQ = 1.0 / (m - 1) if m > 1 else 1.0

    # condition1: разрыв между лучшей и второй по интервалам:
    # требуем Q_left(2) - Q_right(1) >= DQ (консервативная проверка)
    condition1 = False
    if m > 1:
        q1_right = sorted_Q[0, 1]
        q2_left = sorted_Q[1, 0]
        condition1 = (q2_left - q1_right) >= DQ

    # condition2: лучшая по Q должна быть лучшей хотя бы по одному из S или R в пессимистичном виде
    best_idx = int(order[0])
    S_left_best = S[best_idx, 0]
    R_left_best = R[best_idx, 0]
    condition2 = (S_left_best == np.min(S[:, 0])) or (R_left_best == np.min(R[:, 0]))

    best_alternative = best_idx
    if not (condition1 and condition2) and m > 1:
        best_alternative = [int(order[0]), int(order[1])]

    return {
        'ranking': ranking,
        'Q': Q.tolist(),
        'S': S.tolist(),
        'R': R.tolist(),
        'best_alternative': best_alternative,
        'conditions_met': bool(condition1 and condition2),
        'Q_intervals': Q.tolist(),
        'f_star': f_star.tolist(),
        'f_minus': f_minus.tolist(),
        'D_low': D_low.tolist(),
        'D_high': D_high.tolist()
    }

# Пример использования
if __name__ == "__main__":
    matrix = np.array([
        [[0.7, 0.9], [0.6, 0.8], [0.8, 1.0]],
        [[0.5, 0.7], [0.8, 1.0], [0.6, 0.8]],
        [[0.8, 1.0], [0.5, 0.7], [0.7, 0.9]],
        [[0.6, 0.8], [0.7, 0.9], [0.5, 0.7]]
    ])
    weights = [0.3, 0.4, 0.3]
    types = [1, 1, -1]
    res = interval_vikor(matrix, weights, types, v=0.5)

    import pprint
    pprint.pprint(res)


{'D_high': [[0.6000000000000001, 0.8, 1.0],
            [1.0, 0.3999999999999999, 0.0],
            [0.3999999999999999, 1.0, 0.0],
            [0.8, 0.6000000000000001, 0.0]],
 'D_low': [[0.0, 0.0, 1.0],
           [1.0, 0.0, 0.6000000000000001],
           [0.0, 1.0, 0.8],
           [0.0, 0.0, 0.3999999999999999]],
 'Q': [[0.0, 0.8571428571428572],
       [0.24305555555555544, 0.5714285714285714],
       [1.0, 0.7941176470588236],
       [0.0, 0.47899159663865565]],
 'Q_centers': [0.4285714285714286,
               0.40724206349206343,
               0.8970588235294118,
               0.23949579831932782],
 'R': [[0.3, 0.32000000000000006],
       [0.3, 0.3],
       [0.4, 0.4],
       [0.11999999999999997, 0.24000000000000005]],
 'S': [[0.3, 0.8],
       [0.48, 0.45999999999999996],
       [0.64, 0.52],
       [0.11999999999999997, 0.48000000000000004]],
 'best_alternative': [3, 1],
 'conditions_met': False,
 'f_minus': [[0.5, 0.7], [0.5, 0.7], [0.8, 1.0]],
 'f_star': [[0.8, 1.0], [